# Experiments for "Simplified 2D Setting"

### Setup Notebook

Before running this interactive notebook, check the following parameters:
- `ENV`: Select your execution environment `"LOCAL"` (VS Code, Jupyter Notebook) or `"COLAB"` (Google Colab).
- `PLT_INTERACTIVE`: Select whether the plots should be interactive to allow zooming, panning and resizing. In general, this can stay activated.

In [ ]:
#@title { display-mode: "form" }

ENV = "LOCAL" #@param ["LOCAL", "COLAB"]
PLT_INTERACTIVE = True #@param {type:"boolean"}

# setup environment
LOCAL = "LOCAL"
COLAB = "COLAB"
if ENV == COLAB:
    %cd /content
    !git clone https://github.com/danielyxyang/active_reconstruction.git
    %cd active_reconstruction
    !git submodule update --init
    %pip install -q -r requirements.txt
    %pip install -q -r src/utils_ext/requirements.txt
    %cd src
elif ENV == LOCAL:
    %cd ../src

# setup interactive plots
if PLT_INTERACTIVE:
    if ENV == COLAB:
        from google.colab import output
        output.enable_custom_widget_manager()
    %matplotlib widget
else:
    %matplotlib inline

# ensure automatic reload of imported modules
%load_ext autoreload
%autoreload 2


### CUSTOM SETUP ###

if ENV == COLAB:
    # requiring disabled custom widget manager and ipywidgets 7.* to correctly
    # display Tab widget in COLAB, changes to code marked with FIX
    # https://github.com/googlecolab/colabtools/issues/3105
    # https://github.com/googlecolab/colabtools/issues/3571
    %pip install -q "ipywidgets==7.*"

import os
import json
from datetime import datetime
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import ipywidgets as widgets
from tqdm.auto import tqdm
from IPython.display import display, clear_output

import parameters as params
from algorithms.algorithms import ALGORITHMS, TRUE_ALGORITHM
from simulation.plotter import SimulationPlotter, ALGORITHM_COLORS
from simulation.simulation import Simulation, SimulationResults
from simulation.camera import Camera
from simulation.objects import Object, EllipseObject, SquareObject, FlowerObject, PolygonObject
from utils.widgets import KernelSelector, ObjectSelector
from utils_ext.tools import Profiler, build_json_encoder
from utils_ext.widgets import build_widget_outputs

plt.ioff() # prevent figures to be displayed without calling plt.show() or display()
SimulationPlotter.set_interactive(PLT_INTERACTIVE)

### Setup Setting

Optionally, change the parameters of the simplified 2D setting.

In [ ]:
#@title { display-mode: "form" }

#@markdown *All values are specified in `[m]` except for `CAM_FOV`, which is specified in `[deg]`.*

#@markdown world
params.GRID_H = 0.1 # @param {type:"slider", min:0.05, max:1, step:0.05}

#@markdown object
params.OBJ_D_MAX = 8 #@param {type:"number"}
params.OBJ_D_MIN = 2 #@param {type:"number"}

#@markdown camera
params.CAM_D = 10 #@param {type:"number"} 
params.CAM_DOF = 10 #@param {type:"number"}
params.CAM_FOV = 35  #@param {type:"number"}
params.OBS_NOISE = 0.2 #@param {type:"slider", min:0, max:5, step:0.1}

### Run Experiments

In [ ]:
#@title { display-mode: "form" }

if PLT_INTERACTIVE and ENV == COLAB:
    # hack for displaying toolbar of interactive plots in Colab
    html_hack = widgets.HTML("<style> .jupyter-matplotlib-figure { position: relative; } </style>")
    display(html_hack)

def run_evaluation():

    ### DEFINE PARAMETERS ###

    REC_THRESH = 0.95
    OUTPUT = os.path.join(".", "..", "output", "experiments")
    
    # define metrics
    M_RANK = "#"
    M_REC = "reconstruction"
    M_MIS = "missing"
    M_MEA = "measurements"
    M_MEA_THRESH = "measurements{:.0f}".format(REC_THRESH * 100)
    M_REG_AVG = "average regret"
    M_REG_MAX = "max regret"
    M_REG_MIN = "min regret"
    metrics_f = {
        M_REC:        lambda results: results.n_total_final_rel,
        M_MIS:        lambda results: results.n_remaining,
        M_MEA:        lambda results: results.n_measurements,
        M_MEA_THRESH: lambda results: results.n_measurements_upto_thresh(REC_THRESH),
        M_REG_AVG:    lambda results: results.regret_avg,
        M_REG_MAX:    lambda results: results.regret_max,
        M_REG_MIN:    lambda results: results.regret_min,
    }
    metrics_abbrev = {
        M_REC:        "rec",
        M_MIS:        "miss",
        M_MEA:        "mea",
        M_MEA_THRESH: "mea{:.0f}".format(REC_THRESH * 100),
        M_REG_AVG:    "reg",
        M_REG_MAX:    "max reg",
        M_REG_MIN:    "min reg",
    }
    # define ranking
    DESC = -1
    ASC = 1
    ranking = [
        # (M_REG_AVG, ASC), # uncomment to order by regret
        (M_MEA_THRESH, ASC),
        (M_MEA, ASC),
        (M_REC, DESC),
    ]

    ### DEFINE FUNCTIONS ###

    def metric_from_result(object, algorithm, metric):
        if algorithm in results[object]:
            result = results[object][algorithm]
            return metrics_f[metric](result)
        else:
            return np.nan
    
    def compute():
        # display progress bars
        with out["progressbar"]:
            clear_output(wait=True)
            tqdm_objects = tqdm(total=len(selected["objects"]), desc="Objects", bar_format="{desc} |{bar}| {n_fmt}/{total_fmt} [{elapsed}<{remaining}{postfix}]", leave=True)
            tqdm_algorithms = tqdm(total=len(selected["algorithms"]), desc="Algorithms", bar_format="{desc} |{bar}| {n_fmt}/{total_fmt} [{elapsed}<{remaining}{postfix}]", leave=True)
            tqdm_steps = tqdm(total=100, unit="%", desc="Simulation", bar_format="{desc} |{bar}| {n:.1f}{unit} [{elapsed}<{remaining}{postfix}]", leave=False)
        
        # run simulations for all selected algorithms and objects
        with profiler.cm("computation"):
            # loop over objects
            for object in selected["objects"]:
                tqdm_objects.set_postfix(object=object)
                # loop over algorithms
                tqdm_algorithms.reset()
                for algorithm in selected["algorithms"]:
                    tqdm_algorithms.set_postfix(algorithm=algorithm)
                    # initialize simulation
                    simulation = Simulation.build(
                        object=object,
                        camera=Camera(),
                        kernel=selected["kernel"],
                        algorithm=algorithm,
                    )
                    # run simulation
                    tqdm_steps.reset()
                    while not simulation.is_converged():
                        tqdm_steps.set_postfix(iteration=len(simulation.n_marginal) + 1)
                        simulation.step()
                        # update progress bar
                        tqdm_steps.update(simulation.progress() * 100 - tqdm_steps.n)
                    results[object][algorithm] = simulation.results()
                    plot_results(objects=[object])
                    # update progress bar
                    tqdm_algorithms.update(1)
                # update progress bar
                tqdm_objects.update(1)
            # complete progress bar such that it disappears in case of incomplete reconstruction
            tqdm_steps.update(tqdm_steps.total - tqdm_steps.n)
            tqdm_steps.close()
            tqdm_algorithms.close()
            tqdm_objects.close()
    
    def compute_rankings():
        with profiler.cm("computation"):
            # compute table data for rankings
            metrics_ranking = [M_MEA_THRESH, M_MEA, M_REC, M_MIS, M_REG_AVG, M_REG_MAX] # M_RANK automatically included
            df_rankings = {}
            rank_columns, rank_order = zip(*ranking)
            for object in selected["objects"]:
                df_ranking = pd.DataFrame({
                    metric: {
                        algorithm: metric_from_result(object, algorithm, metric)
                        for algorithm in selected["algorithms"] # index
                    }
                    for metric in metrics_ranking # columns
                })
                df_ranking[M_RANK] = (df_ranking
                    .apply(
                        lambda cols: tuple(cols[list(rank_columns)] * list(rank_order)) # turn columns into comparable tuples based on order
                            # remove missing results or reconstruction below threshold
                            if cols.notna().any() and cols[M_REC] >= REC_THRESH
                            else np.nan,
                        axis=1,
                    )
                    .rank(method="dense") # rank without gaps
                )
                df_rankings[object] = df_ranking
            results["_rankings"] = df_rankings

            # compute table data for summary
            metrics_summary = [M_MEA_THRESH, M_MEA, M_REC, M_REG_AVG] # M_RANK automatically included
            metrics_summary_avg = [M_MEA_THRESH, M_MEA, M_REC, M_REG_AVG] # M_RANK automatically included
            df_summary = pd.DataFrame({
                (object, metric): {
                    algorithm: metric_from_result(object, algorithm, metric) if metric != M_RANK else df_rankings[object].loc[algorithm, M_RANK]
                    for algorithm in selected["algorithms"] # index
                }
                for object in selected["objects"] # columns level 0 (outer loop)
                for metric in [M_RANK] + metrics_summary # columns level 1 (inner loop)
            })
            if len(df_rankings) > 0:
                # compute average metrics to summarize summary
                avg_ranking = df_summary.loc[:, pd.IndexSlice[:, M_RANK]].mean(axis=1)
                df_summary.insert(0, ("avg", M_RANK), avg_ranking)
                for i, metric in enumerate(metrics_summary_avg, start=1):
                    if metric == M_MEA or metric == M_MEA_THRESH:
                        # compute number of measurements relative to optimal number of measurements
                        n_measurements = df_summary.loc[:, pd.IndexSlice[:, metric]]
                        n_measurements_opt = df_summary.loc[TRUE_ALGORITHM, pd.IndexSlice[:, metric]]
                        avg_metric = (n_measurements / n_measurements_opt).mean(axis=1)
                    else:
                        avg_metric = df_summary.loc[:, pd.IndexSlice[:, metric]].mean(axis=1)
                    df_summary.insert(i, ("avg", metric), avg_metric)
            results["_summary"] = df_summary

    def plot_results(objects=None):
        if objects is None:
            objects = selected["objects"]

        with profiler.cm("plotting"):
            compute_rankings()
            plot_summary()
            for object in objects:
                plot_object_results(object)

        with out["log"]:
            clear_output(wait=True)
            # print profiling
            profiler.print(["initialization", "computation", "plotting", "display"])
    
    def plot_summary():
        def highlight_metrics(df, style, where):
            df_style = pd.DataFrame("", index=df.index, columns=df.columns)
            for object, algorithm in where:
                for metric in df[[object]].columns.get_level_values(1):
                    df_style.loc[algorithm, (object, metric)] = style
            return df_style
        
        df_summary = results["_summary"]
        if len(df_summary) > 0:
            df_summary = df_summary.sort_values(
                by=[("avg", M_RANK)] + [("avg", metric) for metric, _ in ranking],
                ascending=[True] + [metric_order == ASC for _, metric_order in ranking],
            )
        # apply style
        df_summary_style = df_summary.style
        if len(df_summary) > 0:
            # left-align and color algorithm names
            df_summary_style.applymap_index(lambda index: "text-align: left;", axis="index")
            df_summary_style.applymap_index(lambda index: "color: {};".format(ALGORITHM_COLORS[index]), axis="index")
            # highlight rankings
            df_summary_style.applymap(lambda value: "font-weight: bold;", subset=pd.IndexSlice[:, pd.IndexSlice[:, M_RANK]])
            # highlight TRUE_ALGORITHM
            index_true = list(filter(lambda algorithm: algorithm == TRUE_ALGORITHM, selected["algorithms"]))
            df_summary_style.applymap_index(lambda index: "background-color: rgba(175, 80, 80, 0.25);" if index in index_true else None, axis="index")
            df_summary_style.applymap(lambda value: "background-color: rgba(175, 80, 80, 0.25);", subset=(index_true, df_summary.columns))
            # highlight best reconstructions
            cells_top = [(object, algorithm) 
                for object in selected["objects"] for algorithm in selected["algorithms"]
                if df_summary.loc[algorithm, (object, M_RANK)] == 1
            ]
            df_summary_style.apply(lambda df: highlight_metrics(df, "color: green", cells_top), axis=None)
            # gray out incomplete reconstructions
            cells_incomplete = [(object, algorithm) 
                for object in selected["objects"] for algorithm in selected["algorithms"]
                if df_summary.loc[algorithm, (object, M_REC)] < REC_THRESH
            ]
            df_summary_style.apply(lambda df: highlight_metrics(df, "color: darkgray", cells_incomplete), axis=None)
            # gray out algorithms to be hidden
            index_hide = list(filter(lambda algorithm: algorithm not in select_show_algorithms.value, selected["algorithms"]))
            df_summary_style.applymap_index(lambda index: "color: darkgray;" if index in index_hide else None, axis="index")
            df_summary_style.applymap(lambda value: "color: darkgray;", subset=(index_hide, df_summary.columns))
            # gray out not yet computed results
            df_summary_style.applymap(lambda value: "color: darkgray;" if np.isnan(value) else None)
            # format values
            df_summary_style.format("{:.0%}", na_rep="N/A", subset=pd.IndexSlice[:, pd.IndexSlice[:, M_REC]])
            df_summary_style.format(precision=0, na_rep="N/A", subset=pd.IndexSlice[:, pd.IndexSlice[:, [M_RANK, M_MEA, M_MEA_THRESH]]])
            df_summary_style.format(precision=2, na_rep="N/A", subset=pd.IndexSlice[:, pd.IndexSlice[:, M_REG_AVG]])
            # format values differently in avg
            df_summary_style.format(precision=1, na_rep="N/A", subset=pd.IndexSlice[:, pd.IndexSlice["avg", M_RANK]])
            df_summary_style.format("{:.0%}", na_rep="N/A", subset=pd.IndexSlice[:, pd.IndexSlice["avg", [M_MEA, M_MEA_THRESH]]])
            # abbreviate metric names
            df_summary_style.format_index(lambda metric: metrics_abbrev.get(metric, metric), axis="columns", level=1)

            results["_summary_style"] = df_summary_style

        with profiler.cm("display"):
            with out["tab_summary"]:
                display(df_summary_style, clear=True)

    def plot_object_results(object):
        results_obj = results[object]
        df_ranking = results["_rankings"][object]
        plt_object = plotters[object]["plt_object"]
        plt_total = plotters[object]["plt_total"]
        plt_marginal = plotters[object]["plt_marginal"]
        plt_regret = plotters[object]["plt_regret"]
        out_obj = plotters[object]["out"]

        df_ranking = (df_ranking
            .reset_index(names="algorithm") # make algorithm names to column
            .reindex(columns=[M_RANK, "algorithm", *df_ranking.columns.drop(M_RANK)]) # place ranking column first
            .sort_values(by=M_RANK)
        )
        # apply style
        df_ranking_style = df_ranking.style
        df_ranking_style.hide(axis="index")
        # left-align and color algorithm names
        df_ranking_style.applymap_index(lambda index: "text-align: left;" if index == "algorithm" else None, axis="columns")
        df_ranking_style.applymap(lambda value: "text-align: left;", subset="algorithm")
        df_ranking_style.applymap(lambda value: "color: {};".format(ALGORITHM_COLORS[value]), subset="algorithm")
        # highlight TRUE_ALGORITHM
        index_true = df_ranking.index[df_ranking["algorithm"] == TRUE_ALGORITHM]
        df_ranking_style.applymap(lambda value: "background-color: rgba(175, 80, 80, 0.25);", subset=(index_true, df_ranking.columns))
        # gray out incomplete reconstructions
        index_incomplete = df_ranking.index[df_ranking[M_REC] < REC_THRESH]
        df_ranking_style.applymap(lambda value: "color: darkgray;", subset=(index_incomplete, df_ranking.columns.drop("algorithm")))
        # gray out algorithms to be hidden
        index_hide = df_ranking.index[~df_ranking["algorithm"].isin(select_show_algorithms.value)]
        df_ranking_style.applymap(lambda value: "color: darkgray;", subset=(index_hide, df_ranking.columns))
        # gray out not yet computed results
        index_nan = df_ranking.index[df_ranking[M_REC].isna()]
        df_ranking_style.applymap(lambda value: "color: darkgray;", subset=(index_nan, df_ranking.columns))
        # format values
        df_ranking_style.format("{:.2%}", na_rep="N/A", subset=M_REC)
        df_ranking_style.format(precision=0, na_rep="N/A", subset=[M_RANK, M_MIS, M_MEA, M_MEA_THRESH])
        df_ranking_style.format(precision=2, na_rep="N/A", subset=[M_REG_AVG, M_REG_MAX])
        
        if "_ranking_style" not in results:
            results["_rankings_style"] = {}
        results["_rankings_style"][object] = df_ranking_style

        # plot object
        plt_object.plot_object(object)
        
        # plot relative number of total observations
        max_n_measurements = np.max([result.n_measurements for result in results_obj.values()], initial=10)
        plt_total.axis.set_xlim([0, max_n_measurements + 2])
        plt_total.axis.set_ylim([0, 1.2])
        plt_total.static("n_max", lambda: plt_total.axis.axhline(1, color="red", linestyle="--", linewidth=1))
        plt_total.static("n_thresh", lambda: plt_total.axis.axhline(REC_THRESH, color="red", alpha=0.25, linestyle="--", linewidth=1))
        for algorithm in selected["algorithms"]:
            x = results_obj[algorithm].rounds if algorithm in results_obj else []
            y = results_obj[algorithm].n_total_rel if algorithm in results_obj else []
            plt_total.dynamic_plot("n_total:" + algorithm, x, y, marker="o", markersize=4, color=ALGORITHM_COLORS[algorithm], visible=algorithm in select_show_algorithms.value)
        
        # plot absolute number of marginal observations
        max_n_marginal = np.max([np.max(result.n_marginal) for result in results_obj.values()], initial=10)
        plt_marginal.axis.set_xlim([0, max_n_measurements + 2])
        plt_marginal.axis.set_ylim([0, max_n_marginal * 1.2])
        for algorithm in selected["algorithms"]:
            x = results_obj[algorithm].rounds if algorithm in results_obj else []
            y = results_obj[algorithm].n_marginal if algorithm in results_obj else []
            plt_marginal.dynamic_plot("n_marginal:" + algorithm, x, y, marker="o", markersize=4, color=ALGORITHM_COLORS[algorithm], visible=algorithm in select_show_algorithms.value)
        
        # plot regret
        max_regret = np.max([np.max(result.regret) for result in results_obj.values()], initial=10)
        plt_regret.axis.set_xlim([0, max_n_measurements + 2])
        plt_regret.axis.set_ylim([-2, max_regret * 1.2])
        for algorithm in selected["algorithms"]:
            x = results_obj[algorithm].rounds if algorithm in results_obj else []
            y = results_obj[algorithm].regret if algorithm in results_obj else []
            plt_regret.dynamic_plot("regret:" + algorithm, x, y, marker="o", markersize=4, color=ALGORITHM_COLORS[algorithm], visible=algorithm in select_show_algorithms.value)
        
        with profiler.cm("display"):
            with out_obj["ranking"]:
                display(df_ranking_style, clear=True)
            plt_object.display(out_obj["fig_object"])
            plt_total.display(out_obj["fig_total"])
            plt_marginal.display(out_obj["fig_marginal"])
            plt_regret.display(out_obj["fig_regret"])
    
    def plot_object_preview(obj):
        with profiler.cm("plotting"):
            plt_object.plot_object(obj)
        with profiler.cm("display"):
            plt_object.display(out["fig_object"])
    
    ### DEFINE WIDGET HANDLERS ###

    def observe_object_selector(*args):
        refresh_buttons()
        plot_object_preview(object_selector.value)

    def add_object(*args):
        select_objects.options += (object_selector.value,)
        refresh_buttons()

    def remove_object(*args):
        select_objects.options = list(filter(lambda option: option not in select_objects.value, select_objects.options))
        refresh_buttons()
    
    def move_object_up(*args):
        i = select_objects.index[0]
        options = list(select_objects.options)
        options[i], options[i-1] = options[i-1], options[i]
        select_objects.options = tuple(options)
        select_objects.index = [i-1]
        refresh_buttons()
    
    def move_object_down(*args):
        i = select_objects.index[0]
        options = list(select_objects.options)
        options[i], options[i+1] = options[i+1], options[i]
        select_objects.options = tuple(options)
        select_objects.index = [i+1]
        refresh_buttons()
    
    def observe_select_objects(change):
        refresh_buttons()
        if len(change["new"]) == 1:
            plot_object_preview(change["new"][0])
        elif len(change["new"]) - len(change["old"]) == 1:
            plot_object_preview(set(change["new"]).difference(set(change["old"])).pop())
    
    def observe_select_algorithms(*args):
        refresh_buttons()
        # enforce selection of TRUE_ALGORITHM
        if TRUE_ALGORITHM not in select_algorithms.value:
            select_algorithms.unobserve(observe_select_algorithms, names="value")
            select_algorithms.index = sorted(select_algorithms.index + (select_algorithms.options.index(TRUE_ALGORITHM),))
            select_algorithms.observe(observe_select_algorithms, names="value")

    def refresh_buttons(*args):
        button_obj_add.disabled = object_selector.value in select_objects.options
        button_obj_remove.disabled = len(select_objects.value) == 0
        button_obj_up.disabled = len(select_objects.index) != 1 or select_objects.index[0] == 0
        button_obj_down.disabled = len(select_objects.index) != 1 or select_objects.index[0] == len(select_objects.options) - 1
        if button_start_init.description == "init":
            button_start_init.disabled = len(select_objects.value) == 0 or len(select_algorithms.value) == 0

    def setup(objects, algorithms, kernel):
        # initialize global variables
        nonlocal selected
        selected = dict(
            objects=objects,
            algorithms=algorithms,
            kernel=kernel,
        )
        nonlocal results
        results = {object: {} for object in objects}
        nonlocal plotters
        plotters = {object: {} for object in objects}
        for object, plts in plotters.items():
            plts["plt_object"] = SimulationPlotter(mode="real", figsize=1.5, undecorated=True)
            plts["plt_total"] = SimulationPlotter(figsize=(7, 3), title="Reconstruction Progress ({})".format(object), xlabel="rounds", ylabel="progress")
            plts["plt_marginal"] = SimulationPlotter(figsize=(7, 3), title="Marginal Observations ({})".format(object), xlabel="rounds", ylabel="#points")
            plts["plt_regret"] = SimulationPlotter(figsize=(7, 3), title="Individual Regret ({})".format(object), xlabel="rounds", ylabel="#points")
            plts["out"] = build_widget_outputs(["fig_object", "ranking", "fig_total", "fig_marginal", "fig_regret"])
        
        # initialize widgets
        tab_containers = [tab_results.children[0]]
        # tab_titles = [tab_results.titles[0]]
        tab_titles = [tab_results.get_title(0)] # FIX for compatibility with ipywidgets 7.*
        for object in objects:
            out = plotters[object]["out"]
            tab_containers.append(widgets.VBox([
                widgets.HBox([out["ranking"], out["fig_object"]]),
                out["fig_total"],
                out["fig_marginal"],
                out["fig_regret"],
            ]))
            tab_titles.append(str(object))
        tab_results.children = tab_containers
        # tab_results.titles = tab_titles
        for i, title in enumerate(tab_titles):
            tab_results.set_title(i, title) # FIX for compatibility with ipywidgets 7.*
        select_show_algorithms.unobserve(refresh_plot, names="value")
        select_show_algorithms.options = algorithms
        select_show_algorithms.value = algorithms
        select_show_algorithms.observe(refresh_plot, names="value")

    def init_or_start(*args):
        profiler.reset()
     
        # initialize experiments
        if button_start_init.description == "init":
            button_start_init.description = "start"
            button_start_init.disabled = False
            button_reset.disabled = False
            button_save.disabled = True
            button_load.disabled = True

            setup(
                objects=select_objects.value,
                algorithms=select_algorithms.value,
                kernel=kernel_selector.value,
            )
            plot_results()
        
        # start experiments
        elif button_start_init.description == "start":
            button_start_init.disabled = True
            button_reset.disabled = True
            button_save.disabled = True
            button_load.disabled = True
            compute()
            button_start_init.description = "init"
            button_start_init.disabled = len(select_objects.value) == 0 or len(select_algorithms.value) == 0
            button_reset.disabled = True
            button_save.disabled = False
            button_load.disabled = False
    
    def reset(*args):
        profiler.reset()

        button_start_init.description = "init"
        button_start_init.disabled = len(select_objects.value) == 0 or len(select_algorithms.value) == 0
        button_reset.disabled = True
        button_save.disabled = True
        button_load.disabled = False

        # reset global variables (before resetting widgets)
        nonlocal selected
        selected = dict(objects=[], algorithms=[], kernel=None)
        nonlocal results
        results = {}
        nonlocal plotters
        plotters = {}
        
        # reset widgets
        tab_results.children = [tab_results.children[0]]
        select_show_algorithms.unobserve(refresh_plot, names="value")
        select_show_algorithms.options = []
        select_show_algorithms.value = []
        select_show_algorithms.observe(refresh_plot, names="value")
        plot_results()
            
    def save_results(*args):
        filetag = "_" + textfield_filetag.value if textfield_filetag.value != "" else ""
        filename = "results{}_{}".format(filetag, datetime.now().strftime("%Y%m%d_%H%M%S"))
        filepath = os.path.join(OUTPUT, filename)
        os.makedirs(os.path.dirname(filepath), exist_ok=True)

        results_saved = {str(object): {"object": object.to_json(), "result": result} for object, result in results.items() if not str(object).startswith("_")}
        # save results as json
        with open("{}.json".format(filepath), "w") as file:
            json.dump(results_saved, file, indent=2, default=build_json_encoder([
                (SimulationResults, lambda results: results.to_dict()),
            ]))
            print("Results saved to \"{}.json\".".format(filepath))
        
        df_summary_style = results["_summary_style"]
        # save summary as txt
        with open("{}.txt".format(filepath), "w") as file:
            df_summary_style.to_string(file)
            print("Summary saved to \"{}.txt\".".format(filepath))
        # save summary as html
        with open("{}.html".format(filepath), "w") as file:
            df_summary_style.to_html(file)
            print("Summary saved to \"{}.html\".".format(filepath))
        
        # append summary to html
        filepath_summary = os.path.join(OUTPUT, textfield_filename_summary.value)
        with open("{}.html".format(filepath_summary), "a") as file:
            file.write("\n\n\n<h3>{}</h3>\n".format(filename))
            df_summary_style.to_html(file)
            print("Summary appended to \"{}.html\".".format(filepath_summary))
    
    def load_results(*args):
        profiler.reset()

        button_load.disabled = True
        button_save.disabled = True
        try:
            with profiler.cm("computation"):
                nonlocal results
                filepath = os.path.join(OUTPUT, textfield_filename.value)
                with open("{}.json".format(filepath), "r") as file:
                    results_dict = json.load(file)
                    setup([Object.from_json(data["object"]) for data in results_dict.values()], list(results_dict[list(results_dict.keys())[0]]["result"].keys()), None)
                    for _, data in results_dict.items():
                        object = Object.from_json(data["object"])
                        for algorithm, result in data["result"].items():
                            results[object][algorithm] = SimulationResults.from_dict(result)
                    plot_results()
        finally:
            button_reset.disabled = False
            button_save.disabled = False
            button_load.disabled = False

    def refresh_plot(*args):
        profiler.reset()
        plot_results()
    
    ### INITIALIZATION ###
    
    profiler = Profiler()
    with profiler.cm("initialization"):
        # define global variables
        selected = dict(objects=[], algorithms=[], kernel=None) # store snapshot of configurations after start
        results = {}
        plotters = {}

        # initialize plotters
        plt_object = SimulationPlotter(mode="real", figsize=1, undecorated=True)

        ### SETUP WIDGETS ###

        # define layouts
        layout_select = dict(flex="1 1 auto", max_width="350px", max_height="300px", align_items="stretch")
        layout_button = dict(width="25%")

        # define output widgets
        out = build_widget_outputs(["fig_object", "progressbar", "tab_summary", "log"])

        # define widgets for simulation setup
        object_selector = ObjectSelector()
        object_selector.observe(observe_object_selector, names="value")
        button_obj_add = widgets.Button(description="+", layout=layout_button)
        button_obj_add.on_click(add_object)
        button_obj_remove = widgets.Button(description="-", layout=layout_button)
        button_obj_remove.on_click(remove_object)
        button_obj_up = widgets.Button(description="▲", layout=layout_button)
        button_obj_up.on_click(move_object_up)
        button_obj_down = widgets.Button(description="▼", layout=layout_button)
        button_obj_down.on_click(move_object_down)
        object_selector_and_buttons = widgets.VBox([
            object_selector,
            widgets.HBox([button_obj_add, button_obj_remove, button_obj_up, button_obj_down]),
        ])
        select_objects = widgets.SelectMultiple(description="objects", layout=layout_select)
        select_objects.observe(observe_select_objects, names="value")
        
        kernel_selector = KernelSelector()
        kernel_selector.observe(refresh_buttons, names="value")
        select_algorithms = widgets.SelectMultiple(options=ALGORITHMS, index=[1,4,6,8,10,12,13,15,17,18], description="algorithm", layout=layout_select)
        select_algorithms.observe(observe_select_algorithms, names="value")
        
        button_start_init = widgets.Button(description="init")
        button_start_init.on_click(init_or_start)
        button_reset = widgets.Button(description="reset", disabled=True)
        button_reset.on_click(reset)
        
        display(
            widgets.HBox([object_selector_and_buttons, select_objects, out["fig_object"]]),
            widgets.HBox([kernel_selector, select_algorithms]),
            widgets.HBox([button_start_init, button_reset]),
        )
        
        # display widgets for results
        select_show_algorithms = widgets.SelectMultiple(options=[], layout=dict(width="250px", max_height="300px", align_items="stretch"))
        select_show_algorithms.observe(refresh_plot, names="value")

        textfield_filename = widgets.Text(value="", description="file")
        button_load = widgets.Button(description="load")
        button_load.on_click(load_results)
        textfield_filetag = widgets.Text(value="", description="file tag")
        textfield_filename_summary = widgets.Text(value="summary", description="summary file")
        button_save = widgets.Button(description="save", disabled=True)
        button_save.on_click(save_results)
        tab_results_summary = widgets.VBox([
            out["tab_summary"],
            textfield_filename,
            button_load,
            textfield_filetag,
            textfield_filename_summary,
            button_save,
        ])
        # tab_results = widgets.Tab(children=[tab_results_summary], titles=["summary"])
        tab_results = widgets.Tab(children=[tab_results_summary])
        tab_results.set_title(0, "summary") # FIX for compatibility with ipywidgets 7.*
        
        if ENV == COLAB:
            output.disable_custom_widget_manager() # FIX
        display(
            out["progressbar"],
            widgets.HBox([select_show_algorithms, tab_results]),
            out["log"],
        )
        if ENV == COLAB:
            output.enable_custom_widget_manager() # FIX

        refresh_buttons()

        # add some objects to select_objects
        select_objects.options += (EllipseObject(),)
        select_objects.options += (SquareObject(),)
        select_objects.options += (FlowerObject(),)
        for name in PolygonObject.polygon_names:
            select_objects.options += (PolygonObject.build(name),)
        select_objects.index = [4]
    
    plot_object_preview(object_selector.value)
    plot_results()

plt.close("all")
run_evaluation()